In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
import os, re, sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from yuval_module.paper_clusterer import PaperClusterer
from yuval_module.paper_source import PaperSource

from sklearn.feature_extraction.text import TfidfVectorizer

import py_3.sim_matrix_3 as sim_matrix_3
import py_3.lr_model_3 as lr_model_3
import py_3.db_scan_3 as db_scan_3
import py_4.get_mesh_vec as get_mesh_vec
import py_4.get_cat_vec as get_cat_vec
import py_4.get_names_vec as get_names_vec
import py_4.get_all_features as get_all_features
import py_4.get_co_authors_vec as get_co_authors_vec


from gensim.models import Word2Vec

import metric_eval_2

%matplotlib inline


import utils
from utils import PROJECT_ROOT, DATA_PATH

FILE = "enriched_labeled_dataset_large.json"

In [4]:
if os.path.exists(PROJECT_ROOT + DATA_PATH + FILE):
    print("READING FROM LOCAL")
    if FILE.split(".")[1] == "json":
        df = pd.read_json(PROJECT_ROOT + DATA_PATH + FILE)
    else:
        df = pd.read_csv(PROJECT_ROOT + DATA_PATH + FILE)
    ps = PaperSource()
else:
    print("PULLING FROM S3")
    ps = sim_matrix_3.load_dataset("enriched_labeled_dataset_large")
    df = ps.get_dataset()

df.drop(columns=["last_author_country"],inplace=True)
df.rename(columns={'ORG_STATE':'last_author_country'},inplace=True)

print("FILE PULLED")

PULLING FROM S3
FILE PULLED


### GET THE MESH EMB

In [80]:
#clean up data
df.dropna(subset=['mesh'],inplace=True)
#everything lower_case (already sorted)
df_corpus =  [[re.sub(r"\/.*","",x.lower()) for x in term] for term in df.mesh.values]

In [86]:
model_mesh = Word2Vec(df_corpus, size=64, window=10, min_count=4, workers=4)

In [87]:
model.save("../data/mesh_data/ownmesh2vec.model")

In [88]:
model_dict = {k : model[k] for k in model.wv.index2word}

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


### CO-AUTH EMB

In [93]:
df.dropna(subset=['authors'],inplace=True)

In [231]:
list_of_lists=df.authors.apply(lambda x: [i['name'].lower().split(",") for i in x] )

In [244]:
all_names = [[["_".join((b[0],b[-1].strip()[:1]))][0] for b in x] for x in list_of_lists]

In [246]:
model_names = Word2Vec(all_names, size=64, window=10, min_count=5, workers=4)

In [247]:
model_names.save("../data/owncoauth2vec.model")

In [249]:
model_name_dict = {k : model_names[k] for k in model_names.wv.index2word}

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [257]:
list_of_lists[0]

[['tiwari', ' p'], ['rosen', ' m'], ['madabhushi', ' a']]